In [1]:
import pandas as pd
import numpy as np
from spiketimes.utils import p_adjust
from tqdm.notebook import tqdm
from citalopram_project.load import load_spikes, load_neurons
from citalopram_project.correlations import pairwise_correlation_spikes, shuffled_isi_correlation_test

In [2]:
neurons = load_neurons()
spikes = load_spikes(block_name="pre").merge(
    neurons[["neuron_id", "session_name", "cluster"]]
)


res = []
for session in tqdm(neurons["session_name"].unique()):
    df_spikes = spikes.loc[lambda x: x.session_name == session]
    res.append(
        shuffled_isi_correlation_test(
            df_spikes=df_spikes,
            bin_width=1,
            n_boot=1000,
            rectify=False,
            show_progress=False,
            fillna=0,
        ).assign(session_name=session)
    )

  0%|          | 0/21 [00:00<?, ?it/s]

In [3]:
correlation_test_results = pd.concat(res).assign(p_adj = lambda x: p_adjust(x.p))

In [4]:
from citalopram_project.load import get_data_dir
derived_data_dir = get_data_dir() / "derived"
derived_data_dir.mkdir(exist_ok=True)

correlation_test_results.to_parquet(derived_data_dir / "pre_1s_spont_correlation_test.parquet.gzip", compression="gzip")

pd.read_parquet(derived_data_dir / "pre_1s_spont_correlation_test.parquet.gzip")

,neuron_1,neuron_2,value,neuron_combination,p,p_adj,session_name
17,1069,1070,0.342114,1069_1070,0.000,0.000000,hamilton_01
34,1069,1071,0.223465,1069_1071,0.000,0.000000,hamilton_01
35,1070,1071,-0.012694,1070_1071,1.200,1.319618,hamilton_01
51,1069,1072,-0.034751,1069_1072,0.300,0.389524,hamilton_01
52,1070,1072,-0.209583,1070_1072,0.000,0.000000,hamilton_01
...,...,...,...,...,...,...,...
355,2655,2662,0.464705,2655_2662,0.000,0.000000,chronic_01
356,2657,2662,-0.284572,2657_2662,0.000,0.000000,chronic_01
357,2658,2662,0.398653,2658_2662,0.000,0.000000,chronic_01
358,2659,2662,0.362867,2659_2662,0.000,0.000000,chronic_01


In [12]:
def bin_and_correlate(
    df_spikes,
    binwidth=1,
    spiketimes_col="spiketimes", 
    cell_col="neuron_id",
    t_start=0,
    sigma=0,
    ):
    ...
    df_binned = ifr_by(
        df_spikes,
        spiketimes_col=spiketimes_col,
        spiketrain_col=cell_col,
        sigma=sigma,
        t_start=t_start,
        fs=1/binwidth,
    )
    df_tidy = pairwise_correlation(
        df_binned,
        cell_col=cell_col,
        value_col="ifr", 
        return_tidy=True,
        fillna=0
    )
    return df_tidy

In [15]:
df_list = []
bins = [0.01, 0.1, 0.2, 0.5, 1, 2, 5, 10, 30, 60, 120]
for binwidth in bins:
    for session in tqdm(session_names):
        df1 = spikes.loc[lambda x: x.session_name == session]
        res = bin_and_correlate(df1, binwidth=1)
        df_list.append(res.assign(session_name=session, binwidth=binwidth))

binwidth_res = pd.concat(df_list)


  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

...

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

In [25]:
# binwidth_res.to_parquet(derived_data_dir / "binwidth_spont_correlations.parquet.gzip", compression="gzip")